In [1]:
import numpy as np
import networkx as nx
from dotmotif import Motif, GrandIsoExecutor
import matplotlib.pyplot as plt
from scipy.stats import ranksums
from lsmm_data import LSMMData
import json
from tqdm import tqdm
import scipy.stats as stats
import pandas as pd
# from collections import defaultdict
import pickle

with open('v1dd_data_structures_information.json') as f:
    loaded_json = json.load(f)
my_data = LSMMData.LSMMData(loaded_json)

c:\Users\saika\anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
c:\Users\saika\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
c:\Users\saika\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


Generating Connectome...


100%|██████████| 137706/137706 [03:43<00:00, 616.20it/s]


Generating Connectome...


100%|██████████| 137706/137706 [03:45<00:00, 609.60it/s]


In [86]:
pip install update lsmm_data-0.1.2-py2.py3-none-any.whl

Processing c:\users\saika\downloads\assembly_paper\lsmm_data-0.1.2-py2.py3-none-any.whl
  Attempting uninstall: lsmm-data
    Found existing installation: lsmm_data 0.1.6
    Uninstalling lsmm_data-0.1.6:
      Successfully uninstalled lsmm_data-0.1.6
Note: you may need to restart the kernel to use updated packages.


In [88]:
pip show lsmm_data

Name: lsmm_dataNote: you may need to restart the kernel to use updated packages.

Version: 0.1.2
Summary: 
Home-page: 
Author: 
Author-email: 
License: 
Location: c:\users\saika\anaconda3\lib\site-packages
Requires: 
Required-by: 


In [2]:
data_a = my_data.data
params_a = my_data.params
dirs_a = my_data.dirs
mappings_a = my_data.mappings

In [3]:
# Make a graph
cell_table = data_a['structural']['pre_cell'].copy()
cell_table['connectome_index'] = cell_table.index
post_cell_table = data_a['structural']['post_cell'].copy()
post_cell_table['connectome_index'] = post_cell_table.index

synapse_table = data_a['structural']['synapse']
# adjacency_matrix = data_a['structural']['binary_connectome']

binary_connectome = data_a['structural']['binary_connectome']
summed_size_connectome = data_a['structural']['summed_size_connectome']

pyr_graph = nx.from_numpy_array(binary_connectome, create_using=nx.DiGraph)
import pickle

with open('cell_table.pkl', 'wb') as f:
    pickle.dump(cell_table, f)

with open('post_cell_table.pkl', 'wb') as f:
    pickle.dump(post_cell_table, f)

with open('synapse_table.pkl', 'wb') as f:
    pickle.dump(synapse_table, f)

# # Add edges to the graph
# rows, cols = data_a['structural']['binary_connectome'].shape
# for i in range(rows):
#     for j in range(cols):
#         if data_a['structural']['binary_connectome'][i, j] != 0:  # 0 means no edge
#             pyr_graph.add_edge(f'pre_{i}', f'post_{j}', weight=data_a['structural']['binary_connectome'][i, j])

## Uncomment below to load existing chain motif analysis results
# two_chain_results_array = np.load('all_cell_two_chain_results_array.npy')
# three_chain_results_array = np.load('all_cell_three_chain_results_array.npy')
# four_chain_results_array = np.load('all_cell_four_chain_results_array.npy')

# Or uncomment below to generate new motif analysis results (Can take quite a while on larger graphs)
# Motif Analysis with DotMotif: 2 Chain, All Pyr
executor = GrandIsoExecutor(graph=pyr_graph)
chain_defs = Motif("""
                A -> B
                B -> C
              """)

chain_results = executor.find(chain_defs)

# Filter the chain results for chains from an excitatory cell to an inhibitory cell to an excitatory cell
# filtered_chain_results = []
# for chain in chain_results:
#     if cell_table.iloc[chain['A'].split('_')[1]]. and cell_table.iloc[chain['A'].split('_')[1]] chain['C'].split('_')[0] == 'post':
#         filtered_chain_results.append(chain)


two_chain_results_array = np.array([list(c.values()) for c in tqdm(chain_results)])
# removed_pre_post_strings = np.zeros_like(two_chain_results_array)
# for i, c in enumerate(chain_results):
#     for j, n in enumerate(c.values()):
#         removed_pre_post_strings[i, j] = int(n.split('_')[1])

#two_chain_results_array = np.array(removed_pre_post_strings).astype(int)

# two_chain_results_array = np.array([list(c.values()) for c in tqdm(chain_results)])

#assembly_chain_results_arrays = [two_chain_results_array]

chain_count_string_array = ['pyr_cell_2chain']
individual_assembly_indexes = [mappings_a['connectome_indexes_by_assembly'][f'A {i}'] for i in range(1,16)]
individual_post_assembly_indexes = [mappings_a['post_connectome_indexes_by_assembly'][f'A {i}'] for i in range(1,16)]

coregistered_post_cell_indexes = mappings_a['assemblies_by_post_connectome_index'].keys()
coregistered_cell_indexes = mappings_a['assemblies_by_connectome_index'].keys()
no_a_cell_indexes = mappings_a['connectome_indexes_by_assembly']['No A']
no_a_post_cell_indexes = mappings_a['post_connectome_indexes_by_assembly']['No A']
pooled_assembly_indexes = list(set(coregistered_cell_indexes) - set(no_a_cell_indexes))
pooled_assembly_post_indexes = list(set(coregistered_post_cell_indexes) - set(no_a_post_cell_indexes))

assembly_to_root_ids = mappings_a['pt_root_ids_by_assembly']
assembly_root_ids_set = set(mappings_a['assemblies_by_pt_root_id'].keys())

# Filter synapses_table to only synapses between two assembly cells (including No A)
synapses_df = synapse_table[synapse_table['pre_pt_root_id'].isin(assembly_root_ids_set)]   #gets assembly cells based on pre-pt-root-id
synapses_df = synapses_df[synapses_df['post_pt_root_id'].isin(assembly_root_ids_set)]    #takes the dict with pre assembly cells and further filters for post pt-root-id 

# Filter cell tables to only assembly cells
assembly_cell_table = cell_table[cell_table['pt_root_id'].isin(assembly_root_ids_set)]
assembly_post_cell_table = post_cell_table[post_cell_table['pt_root_id'].isin(assembly_root_ids_set)]

assembly_pre_root_ids = set(assembly_cell_table['pt_root_id'].values)
assembly_post_root_ids = set(assembly_post_cell_table['pt_root_id'].values)
assembly_all_root_ids = assembly_pre_root_ids | assembly_post_root_ids


100%|██████████| 6052393/6052393 [00:08<00:00, 736071.41it/s] 


In [4]:
x = synapse_table[synapse_table['pre_pt_root_id'].isin(assembly_root_ids_set)]
x

,id,pre_pt_root_id,post_pt_root_id,pt_position_x,pt_position_y,pt_position_z,size,pt_position_x_trafo,pt_position_y_trafo,pt_position_z_trafo,ctr_pt_position
15,385871060,864691132748334211,864691132713081093,803072.7,650714.8,237465.0,1147,-85117.255514,306869.009066,-47134.439571,"[-85117.25551443589, 306869.0090664734, -47134..."
22,400682910,864691132841768198,864691132713081093,805041.8,712988.8,297090.0,1300,-97751.777724,384323.506911,-6662.605649,"[-97751.77772377743, 384323.5069107702, -6662...."
47,354037317,864691132712062021,864691132713081093,752778.2,656806.4,257040.0,911,-137085.302904,318360.625595,-29681.003831,"[-137085.30290407952, 318360.6255949742, -2968..."
51,378751644,864691132734110999,864691132713081093,798513.7,667796.5,248580.0,1574,-92622.696310,326565.622821,-40602.965744,"[-92622.69631006179, 326565.622821283, -40602...."
63,468573544,864691132678493470,864691132937982091,913342.3,728004.4,305865.0,815,6705.538694,401075.747886,-2701.409404,"[6705.53869431683, 401075.74788560363, -2701.4..."
...,...,...,...,...,...,...,...,...,...,...,...
137519,391293053,864691132921491681,864691132655536611,819756.7,632711.6,170910.0,1164,-63588.495897,271251.262965,-107688.956019,"[-63588.49589710794, 271251.2629645617, -10768..."
137545,423870350,864691132921491681,864691132688960776,860797.4,727364.2,223425.0,1875,-40521.812155,378005.434125,-81267.273699,"[-40521.812154830746, 378005.434125318, -81267..."
137547,377128104,864691132747994573,864691132689099528,802587.7,727529.1,265095.0,3963,-101461.016171,389937.531399,-41561.201626,"[-101461.01617127818, 389937.5313992617, -4156..."
137634,373066345,864691132921491681,864691132655931504,766096.3,717499.3,262665.0,308,-135591.979384,379517.190261,-40926.478717,"[-135591.97938415446, 379517.19026121864, -409..."


In [5]:
mappings_a.keys()

dict_keys(['functional_indexes_by_assembly', 'assemblies_by_functional_index', 'pt_root_id_to_connectome_index', 'connectome_index_to_root_id', 'pt_root_id_to_post_connectome_index', 'post_connectome_index_to_root_id', 'roi_id_to_pt_root_id', 'pt_root_id_to_roi_id', 'roi_id_to_functional_index', 'functional_index_to_roi_id', 'assemblies_by_roi_id', 'assemblies_by_pt_root_id', 'assemblies_by_connectome_index', 'assemblies_by_post_connectome_index', 'connectome_indexes_by_assembly', 'post_connectome_indexes_by_assembly', 'pt_root_ids_by_assembly', 'roi_ids_by_assembly'])

### Alternate logic for comparison with Julian's code - Normalization by total chain length

In [6]:
inhibitory_cells =cell_table[cell_table['classification_system'] == 'inhibitory']
inhibitory_cells_root_ids_set = set(inhibitory_cells['pt_root_id'].values)

synapses_df_pre_assembly = synapse_table[synapse_table['pre_pt_root_id'].isin(assembly_root_ids_set)]   #gets assembly cells based on pre-pt-root-id
synapses_df_inhibitory_pre = synapse_table[synapse_table['pre_pt_root_id'].isin(inhibitory_cells_root_ids_set)]

synapses_df_pre_assembly_to_inhibitory = synapses_df_pre_assembly[synapses_df_pre_assembly['post_pt_root_id'].isin(inhibitory_cells_root_ids_set)]

synapses_df_inhibitory_post = synapse_table[synapse_table['post_pt_root_id'].isin(inhibitory_cells_root_ids_set)]
synapses_df_post_assembly = synapse_table[synapse_table['post_pt_root_id'].isin(assembly_root_ids_set)]

synapses_df_inhibitory_to_post_assembly = synapses_df_post_assembly[synapses_df_post_assembly['pre_pt_root_id'].isin(inhibitory_cells_root_ids_set)]

excitatory_cells =cell_table[cell_table['classification_system'] == 'excitatory']
excitatory_cells_root_ids_set = set(excitatory_cells['pt_root_id'].values)

synapses_df_pre_assembly_to_excitatory = synapses_df_pre_assembly[synapses_df_pre_assembly['post_pt_root_id'].isin(excitatory_cells_root_ids_set)]
synapses_df_excitatory_to_post_assembly = synapses_df_post_assembly[synapses_df_post_assembly['pre_pt_root_id'].isin(excitatory_cells_root_ids_set)]

# Prep the sets


#W_ikj - j: pre-cell, k: middle cell, i: post-cell

# Collect w and s for inhibitory - k is inhibitory cell
# corresponds to w_kj in formula (1)
w_jk = {}   #PSD
s_jk = {}   #Synaptic count 
b_jk = {}   #Binary count  or connectivity - either 1 or 0

# corresponds to w_ik in formula (1)
w_ki = {}   #PSD
s_ki = {}   #Synaptic count 
b_ki = {}   #Binary count  or connectivity - either 1 or 0

# Collect w and s for excitatory - k is excitatory cell
w_jkE = {}   #PSD
s_jkE = {}   #Synaptic count 
b_jkE = {}   #Binary count  or connectivity - either 1 or 0

w_kiE = {}   #PSD
s_kiE = {}   #Synaptic count 
b_kiE = {}   #Binary count  or connectivity - either 1 or 0


for pre in assembly_pre_root_ids:
    for middle in inhibitory_cells_root_ids_set:
        w_jk[(pre, middle)] = 0
        s_jk[(pre, middle)] = 0
        b_jk[(pre, middle)] = 0


for middle in inhibitory_cells_root_ids_set:
    for post in assembly_post_root_ids:
        w_ki[(middle, post)] = 0
        s_ki[(middle, post)] = 0
        b_ki[(middle, post)] = 0

# get the PSD from pre-cell j to middle cell k 
for i, row in synapses_df_pre_assembly_to_inhibitory.iterrows():   #check how df.iterrows looks
    pre = row['pre_pt_root_id']
    middle = row['post_pt_root_id']
    w_jk[(pre, middle)] += row['size']
    s_jk[(pre, middle)] += 1
    b_jk[(pre, middle)] = 1

# get the PSD from middle cell k to post-cell i
for i, row in synapses_df_inhibitory_to_post_assembly.iterrows():   #check how df.iterrows looks
    middle = row['pre_pt_root_id']
    post = row['post_pt_root_id']
    w_ki[(middle, post)] += row['size']
    s_ki[(middle, post)] += 1
    b_ki[(middle, post)] = 1

# repeat the same for excitatory middle cell
for pre in assembly_pre_root_ids:
    for middle in excitatory_cells_root_ids_set:
        w_jkE[(pre, middle)] = 0
        s_jkE[(pre, middle)] = 0
        b_jkE[(pre, middle)] = 0

for middle in excitatory_cells_root_ids_set:
    for post in assembly_post_root_ids:
        w_kiE[(middle, post)] = 0
        s_kiE[(middle, post)] = 0
        b_kiE[(middle, post)] = 0

for i, row in synapses_df_pre_assembly_to_excitatory.iterrows():   #check how df.iterrows looks
    pre = row['pre_pt_root_id']
    middle = row['post_pt_root_id']
    w_jkE[(pre, middle)] += row['size']
    s_jkE[(pre, middle)] += 1
    b_jkE[(pre, middle)] = 1


for i, row in synapses_df_excitatory_to_post_assembly.iterrows():   #check how df.iterrows looks
    middle = row['pre_pt_root_id']
    post = row['post_pt_root_id']
    w_kiE[(middle, post)] += row['size']
    s_kiE[(middle, post)] += 1
    b_kiE[(middle, post)] = 1

# Split out assemblies and no_a
assembly_names = set(assembly_to_root_ids.keys()) - set(['No A'])
A_invert = {assembly: set(assembly_to_root_ids[assembly]) for assembly in assembly_names}
no_A = set(assembly_to_root_ids['No A'])
all_coregistered_root_ids = mappings_a['assemblies_by_pt_root_id'].keys()
assembly_root_ids_excluding_no_A = set(all_coregistered_root_ids) - no_A
A = {pt_root_id: set(mappings_a['assemblies_by_pt_root_id'][pt_root_id]) for pt_root_id in assembly_all_root_ids if 'No A' not in mappings_a['assemblies_by_pt_root_id'][pt_root_id]}

# C functions - for getting the different test conditions
def shared(pre, post, A):
    try:
        return not A[pre].isdisjoint(A[post])
    except KeyError:
        return False

def disjoint(pre, post, A):
    try:
        return A[pre].isdisjoint(A[post])
    except KeyError:
        return False

def shared_no_a(pre, post, A):
    return (pre in no_A) and (post in no_A)
    
def no_a_a(pre, post, A):
    return (pre in no_A) and (post not in no_A)

def a_no_a(pre, post, A):
    return (pre not in no_A) and (post in no_A)

C = [shared, disjoint, shared_no_a, no_a_a, a_no_a]

**Outbound connectivity**

In [7]:



classification_map = cell_table.set_index('connectome_index')['classification_system'].to_dict() # Map to tell us if its Excitatory or Inhibitory 


# Calculate the weighted summed values

backup_assembly_pre_root_ids = assembly_pre_root_ids
backup_assembly_post_root_ids = assembly_post_root_ids




W_out_feed_forward_inhibition_chains = {c.__name__: {} for c in C}
W_out_excitatory_chains = {c.__name__: {} for c in C}


for cond_function in C:
    for j in assembly_pre_root_ids:
        # Exclude the current pre-synaptic cell from the post-synaptic candidate set
        normalizing_factor = 0
        post_root_ids = backup_assembly_post_root_ids - set([j])
        for i in post_root_ids:
            # Check if pre-synaptic cell 'j' satisfies the condition with post-synaptic cell 'i'
            if cond_function(j, i, A) and j != i:
                pre_idx = mappings_a['pt_root_id_to_connectome_index'][j]
                post_idx = mappings_a['pt_root_id_to_connectome_index'][i]

                # Filter chains where cell A in chain matches pre_idx and cell C matches post_idx
                temporary_chain_results = two_chain_results_array[(two_chain_results_array[:, 0] == pre_idx) & (two_chain_results_array[:, 2] == post_idx)]

                # Add to the normalizing factor the number of realized chains between pre_cell (position A in chain) and post cell (position C in chain)
                normalizing_factor += len(temporary_chain_results)

                # Continue if no matches are found
                if len(temporary_chain_results) == 0:
                    continue

                for chain in temporary_chain_results:  # Summing over possible intermediate nodes (k's)
                    cell_a, cell_b, cell_c = chain

                    # Classification of the intermediate node (cell B)
                    if classification_map[cell_b] == 'inhibitory':
                        k = mappings_a['connectome_index_to_root_id'][cell_b]
                        W_out_feed_forward_inhibition_chains[cond_function.__name__][j] = (W_out_feed_forward_inhibition_chains[cond_function.__name__].get(j, 0) +
                        (w_jk[(j, k)]* w_ki[(k,i)]) )

                    elif classification_map[cell_b] == 'excitatory':
                        k = mappings_a['connectome_index_to_root_id'][cell_b]
                        W_out_excitatory_chains[cond_function.__name__][j] = (
                            W_out_excitatory_chains[cond_function.__name__].get(j, 0) +
                            (w_jkE[(j,k)]*w_kiE[(k,i)])
                        )
                        
        if j in W_out_feed_forward_inhibition_chains[cond_function.__name__]:
           W_out_feed_forward_inhibition_chains[cond_function.__name__][j] /= normalizing_factor

        if j in W_out_excitatory_chains[cond_function.__name__]:
            W_out_excitatory_chains[cond_function.__name__][j] /= normalizing_factor

                    

In [8]:
W_out_feed_forward_inhibition_chains

{'shared': {864691132807412738: 3464014.1912751677,
  864691132853487363: 6230415.198924731,
  864691132834513288: 2711447.146496815,
  864691132834925960: 3607745.867647059,
  864691132636193294: 3748049.9831460672,
  864691132721900430: 2614292.4246153845,
  864691132771662867: 5484427.110169492,
  864691132577861908: 3345062.606557377,
  864691132734110231: 2459294.9,
  864691132734110999: 3086211.8,
  864691132738257819: 4037475.42010772,
  864691132678493470: 4143772.969907407,
  864691132770059681: 4784253.751552795,
  864691132741590308: 5677303.540229885,
  864691132635895079: 4582167.17535545,
  864691132660826793: 3378548.108280255,
  864691132720041642: 4200799.071428572,
  864691132604748205: 6373719.0195488725,
  864691132731808307: 4395966.9382716045,
  864691132841658548: 4202105.4543379,
  864691132595467831: 6773903.525,
  864691132851739966: 3016538.0285714287,
  864691132598347327: 5000774.457831325,
  864691132598451263: 4394329.593103448,
  864691132712062021: 5047

In [9]:
# Outbound Summed PSDs

print('\n\nSUMMED OUTBOUND PSDS for Feed-Forward Inhibitory CHAINS')
for c1 in C:
    for c2 in C:
        if c1.__name__ != c2.__name__:
            print(f'\n{c1.__name__} vs {c2.__name__}')
            result = stats.ranksums(list(W_out_feed_forward_inhibition_chains[c1.__name__].values()), list(W_out_feed_forward_inhibition_chains[c2.__name__].values()), alternative='greater')

            print('Greater:')
            print(result)
            if result.pvalue < 0.05:
                plt.figure()
                plt.boxplot([list(W_out_feed_forward_inhibition_chains[c1.__name__].values()), list(W_out_feed_forward_inhibition_chains[c2.__name__].values())])
                plt.title(f'OUTBOUND SUMMED PSDS: {c1.__name__} vs {c2.__name__}')



SUMMED OUTBOUND PSDS for Feed-Forward Inhibitory CHAINS

shared vs disjoint
Greater:
RanksumsResult(statistic=-0.5324415876345198, pvalue=0.702789905971789)

shared vs shared_no_a
Greater:
RanksumsResult(statistic=-0.14876789971477894, pvalue=0.5591316094807647)

shared vs no_a_a
Greater:
RanksumsResult(statistic=-0.27628324232744655, pvalue=0.6088347370593239)

shared vs a_no_a
Greater:
RanksumsResult(statistic=-0.05816588772477947, pvalue=0.523191753824342)

disjoint vs shared
Greater:
RanksumsResult(statistic=0.5324415876345198, pvalue=0.297210094028211)

disjoint vs shared_no_a
Greater:
RanksumsResult(statistic=0.22416326513328616, pvalue=0.4113151331526233)

disjoint vs no_a_a
Greater:
RanksumsResult(statistic=0.08151391459392224, pvalue=0.4675166297287648)

disjoint vs a_no_a
Greater:
RanksumsResult(statistic=0.5311750371111018, pvalue=0.2976487448662677)

shared_no_a vs shared
Greater:
RanksumsResult(statistic=0.14876789971477894, pvalue=0.44086839051923526)

shared_no_a vs di

In [62]:
# Outbound Summed PSDs

print('\n\nSUMMED OUTBOUND PSDS for Excitatory CHAINS')
for c1 in C:
    for c2 in C:
        if c1.__name__ != c2.__name__:
            print(f'\n{c1.__name__} vs {c2.__name__}')
            result = stats.ranksums(list(W_out_excitatory_chains[c1.__name__].values()), list(W_out_excitatory_chains[c2.__name__].values()), alternative='greater')
            print('Greater:')
            print(result)
            if result.pvalue < 0.05:
                plt.figure()
                plt.boxplot([list(W_out_excitatory_chains[c1.__name__].values()), list(W_out_excitatory_chains[c2.__name__].values())])
                plt.title(f'OUTBOUND SUMMED PSDS: {c1.__name__} vs {c2.__name__}')



SUMMED OUTBOUND PSDS for Excitatory CHAINS

shared vs disjoint
Greater:
RanksumsResult(statistic=1.097293968840645, pvalue=0.13625645326064317)

shared vs shared_no_a
Greater:
RanksumsResult(statistic=0.7566444492037343, pvalue=0.22463145196248752)

shared vs no_a_a
Greater:
RanksumsResult(statistic=0.2566635011696727, pvalue=0.39871927483755554)

shared vs a_no_a
Greater:
RanksumsResult(statistic=-0.2372526162433467, pvalue=0.5937695902756877)

disjoint vs shared
Greater:
RanksumsResult(statistic=-1.097293968840645, pvalue=0.8637435467393568)

disjoint vs shared_no_a
Greater:
RanksumsResult(statistic=-0.13832083379312204, pvalue=0.5550065696062781)

disjoint vs no_a_a
Greater:
RanksumsResult(statistic=-0.4126536306249159, pvalue=0.660069798859094)

disjoint vs a_no_a
Greater:
RanksumsResult(statistic=-1.208131743471013, pvalue=0.8865017043252169)

shared_no_a vs shared
Greater:
RanksumsResult(statistic=-0.7566444492037343, pvalue=0.7753685480375125)

shared_no_a vs disjoint
Greater:

**Inbound connectivity**

In [10]:
classification_map = cell_table.set_index('connectome_index')['classification_system'].to_dict() # Map to tell us if its Excitatory or Inhibitory 


# Calculate the weighted summed values

backup_assembly_pre_root_ids = assembly_pre_root_ids
backup_assembly_post_root_ids = assembly_post_root_ids




W_in_feed_forward_inhibition_chains = {c.__name__: {} for c in C}
W_in_excitatory_chains = {c.__name__: {} for c in C}


for cond_function in C:
    for i in assembly_post_root_ids:
        # Exclude the current pre-synaptic cell from the post-synaptic candidate set
        normalizing_factor = 0
        pre_root_ids = backup_assembly_pre_root_ids - set([i])
        for j in pre_root_ids:
            # Check if pre-synaptic cell 'j' satisfies the condition with post-synaptic cell 'i'
            if cond_function(j, i, A) and j != i:
                pre_idx = mappings_a['pt_root_id_to_connectome_index'][j]
                post_idx = mappings_a['pt_root_id_to_connectome_index'][i]

                # Filter chains where cell A in chain matches pre_idx and cell C matches post_idx
                temporary_chain_results = two_chain_results_array[(two_chain_results_array[:, 0] == pre_idx) & (two_chain_results_array[:, 2] == post_idx)]

                # Add to the normalizing factor the number of realized chains between pre_cell (position A in chain) and post cell (position C in chain)
                normalizing_factor += len(temporary_chain_results)

                # Continue if no matches are found
                if len(temporary_chain_results) == 0:
                    continue

                for chain in temporary_chain_results:  # Summing over possible intermediate nodes (k's)
                    cell_a, cell_b, cell_c = chain

                    # Classification of the intermediate node (cell B)
                    if classification_map[cell_b] == 'inhibitory':
                        k = mappings_a['connectome_index_to_root_id'][cell_b]
                        W_in_feed_forward_inhibition_chains[cond_function.__name__][i] = (W_in_feed_forward_inhibition_chains[cond_function.__name__].get(i, 0) +
                        (w_jk[(j, k)]* w_ki[(k,i)]) )

                    elif classification_map[cell_b] == 'excitatory':
                        k = mappings_a['connectome_index_to_root_id'][cell_b]
                        W_in_excitatory_chains[cond_function.__name__][i] = (
                            W_in_excitatory_chains[cond_function.__name__].get(i, 0) +
                            (w_jkE[(j,k)]*w_kiE[(k,i)])
                        )
                        
        if i in W_in_feed_forward_inhibition_chains[cond_function.__name__]:
           W_in_feed_forward_inhibition_chains[cond_function.__name__][i] /= normalizing_factor

        if i in W_in_excitatory_chains[cond_function.__name__]:
            W_in_excitatory_chains[cond_function.__name__][i] /= normalizing_factor


In [11]:
# Inbound Summed PSDs

print('\n\nSUMMED INBOUND PSDS for Feed-Forward Inhibitory CHAINS')
for c1 in C:
    for c2 in C:
        if c1.__name__ != c2.__name__:
            print(f'\n{c1.__name__} vs {c2.__name__}')
            result = stats.ranksums(list(W_in_feed_forward_inhibition_chains[c1.__name__].values()), list(W_in_feed_forward_inhibition_chains[c2.__name__].values()), alternative='greater')

            print('Greater:')
            print(result)
            if result.pvalue < 0.05:
                plt.figure()
                plt.boxplot([list(W_in_feed_forward_inhibition_chains[c1.__name__].values()), list(W_in_feed_forward_inhibition_chains[c2.__name__].values())])
                plt.title(f'OUTBOUND SUMMED PSDS: {c1.__name__} vs {c2.__name__}')



SUMMED INBOUND PSDS for Feed-Forward Inhibitory CHAINS

shared vs disjoint
Greater:
RanksumsResult(statistic=-0.5324415876345198, pvalue=0.702789905971789)

shared vs shared_no_a
Greater:
RanksumsResult(statistic=-0.14876789971477894, pvalue=0.5591316094807647)

shared vs no_a_a
Greater:
RanksumsResult(statistic=0.2908294386238974, pvalue=0.38559088463322694)

shared vs a_no_a
Greater:
RanksumsResult(statistic=-0.38254602783800296, pvalue=0.6489718025365293)

disjoint vs shared
Greater:
RanksumsResult(statistic=0.5324415876345198, pvalue=0.297210094028211)

disjoint vs shared_no_a
Greater:
RanksumsResult(statistic=0.3056771797272084, pvalue=0.37992522504711745)

disjoint vs no_a_a
Greater:
RanksumsResult(statistic=0.8248327812050443, pvalue=0.20473326644441436)

disjoint vs a_no_a
Greater:
RanksumsResult(statistic=-0.14264935053936392, pvalue=0.5567164400461241)

shared_no_a vs shared
Greater:
RanksumsResult(statistic=0.14876789971477894, pvalue=0.44086839051923526)

shared_no_a vs d

In [12]:
# Inbound Summed PSDs

print('\n\nSUMMED INBOUND PSDS for Excitatory CHAINS')
for c1 in C:
    for c2 in C:
        if c1.__name__ != c2.__name__:
            print(f'\n{c1.__name__} vs {c2.__name__}')
            result = stats.ranksums(list(W_in_excitatory_chains[c1.__name__].values()), list(W_in_excitatory_chains[c2.__name__].values()), alternative='greater')
            print('Greater:')
            print(result)
            if result.pvalue < 0.05:
                plt.figure()
                plt.boxplot([list(W_in_excitatory_chains[c1.__name__].values()), list(W_in_excitatory_chains[c2.__name__].values())])
                plt.title(f'OUTBOUND SUMMED PSDS: {c1.__name__} vs {c2.__name__}')



SUMMED INBOUND PSDS for Excitatory CHAINS

shared vs disjoint
Greater:
RanksumsResult(statistic=0.19086050424303608, pvalue=0.42431744216601935)

shared vs shared_no_a
Greater:
RanksumsResult(statistic=0.09413875735476426, pvalue=0.46249946657689583)

shared vs no_a_a
Greater:
RanksumsResult(statistic=0.3180418855547094, pvalue=0.3752265825245682)

shared vs a_no_a
Greater:
RanksumsResult(statistic=-0.38620062674037703, pvalue=0.6503259551929315)

disjoint vs shared
Greater:
RanksumsResult(statistic=-0.19086050424303608, pvalue=0.5756825578339806)

disjoint vs shared_no_a
Greater:
RanksumsResult(statistic=0.11230588859277421, pvalue=0.45529043646095624)

disjoint vs no_a_a
Greater:
RanksumsResult(statistic=0.3811319673727111, pvalue=0.3515526639307205)

disjoint vs a_no_a
Greater:
RanksumsResult(statistic=-0.42927285527410686, pvalue=0.6661376656507051)

shared_no_a vs shared
Greater:
RanksumsResult(statistic=-0.09413875735476426, pvalue=0.5375005334231042)

shared_no_a vs disjoint
G

## Revised logic with normalization based on excitatory and inhibitory cells

In [13]:

classification_map = cell_table.set_index('connectome_index')['classification_system'].to_dict() # Map to tell us if its Excitatory or Inhibitory 


# Calculate the weighted summed values

backup_assembly_pre_root_ids = assembly_pre_root_ids
backup_assembly_post_root_ids = assembly_post_root_ids




W_out_feed_forward_inhibition_chains = {c.__name__: {} for c in C}
W_out_excitatory_chains = {c.__name__: {} for c in C}


for cond_function in C:
    for j in assembly_pre_root_ids:
        # Exclude the current pre-synaptic cell from the post-synaptic candidate set
        normalizing_factor = 0
        normalizing_factor_excitatory_middle_cell = 0
        normalizing_factor_inhibitory_middle_cell = 0
        post_root_ids = backup_assembly_post_root_ids - set([j])
        for i in post_root_ids:
            # Check if pre-synaptic cell 'j' satisfies the condition with post-synaptic cell 'i'
            if cond_function(j, i, A) and j != i:
                pre_idx = mappings_a['pt_root_id_to_connectome_index'][j]
                post_idx = mappings_a['pt_root_id_to_connectome_index'][i]

                # Filter chains where cell A in chain matches pre_idx and cell C matches post_idx
                temporary_chain_results = two_chain_results_array[(two_chain_results_array[:, 0] == pre_idx) & (two_chain_results_array[:, 2] == post_idx)]

                # Add to the normalizing factor the number of realized chains between pre_cell (position A in chain) and post cell (position C in chain)
                normalizing_factor += len(temporary_chain_results)

                # Continue if no matches are found
                if len(temporary_chain_results) == 0:
                    continue

                for chain in temporary_chain_results:  # Summing over possible intermediate nodes (k's)
                    cell_a, cell_b, cell_c = chain

                    # Classification of the intermediate node (cell B)
                    if classification_map[cell_b] == 'inhibitory':
                        normalizing_factor_inhibitory_middle_cell += 1
                        k = mappings_a['connectome_index_to_root_id'][cell_b]
                        W_out_feed_forward_inhibition_chains[cond_function.__name__][j] = (W_out_feed_forward_inhibition_chains[cond_function.__name__].get(j, 0) +
                        (w_jk[(j, k)]* w_ki[(k,i)]) )

                    elif classification_map[cell_b] == 'excitatory':
                        normalizing_factor_excitatory_middle_cell +=1
                        k = mappings_a['connectome_index_to_root_id'][cell_b]
                        W_out_excitatory_chains[cond_function.__name__][j] = (
                            W_out_excitatory_chains[cond_function.__name__].get(j, 0) +
                            (w_jkE[(j,k)]*w_kiE[(k,i)])
                        )
                        
        if j in W_out_feed_forward_inhibition_chains[cond_function.__name__]:
           print("Inhibitory count",[j, normalizing_factor_inhibitory_middle_cell, normalizing_factor])
           W_out_feed_forward_inhibition_chains[cond_function.__name__][j] /= normalizing_factor

        if j in W_out_excitatory_chains[cond_function.__name__]:
            print("Excitatory count", [j, normalizing_factor_excitatory_middle_cell, normalizing_factor])
            W_out_excitatory_chains[cond_function.__name__][j] /= normalizing_factor

Inhibitory count [864691132807412738, 298, 298]
Inhibitory count [864691132853487363, 184, 186]
Excitatory count [864691132853487363, 2, 186]
Inhibitory count [864691132834513288, 156, 157]
Excitatory count [864691132834513288, 1, 157]
Inhibitory count [864691132834925960, 136, 136]
Inhibitory count [864691132636193294, 353, 356]
Excitatory count [864691132636193294, 3, 356]
Inhibitory count [864691132721900430, 308, 325]
Excitatory count [864691132721900430, 17, 325]
Inhibitory count [864691132771662867, 235, 236]
Excitatory count [864691132771662867, 1, 236]
Inhibitory count [864691132577861908, 121, 122]
Excitatory count [864691132577861908, 1, 122]
Inhibitory count [864691132734110231, 10, 10]
Inhibitory count [864691132734110999, 446, 455]
Excitatory count [864691132734110999, 9, 455]
Inhibitory count [864691132738257819, 553, 557]
Excitatory count [864691132738257819, 4, 557]
Inhibitory count [864691132678493470, 431, 432]
Excitatory count [864691132678493470, 1, 432]
Inhibitory 

In [14]:
#######---- OUTBOUND CONNECTIVITY-----###########

classification_map = cell_table.set_index('connectome_index')['classification_system'].to_dict() # Map to tell us if its Excitatory or Inhibitory 


# Calculate the weighted summed values

backup_assembly_pre_root_ids = assembly_pre_root_ids
backup_assembly_post_root_ids = assembly_post_root_ids




W_out_feed_forward_inhibition_chains = {c.__name__: {} for c in C}
W_out_excitatory_chains = {c.__name__: {} for c in C}


for cond_function in C:
    for j in assembly_pre_root_ids:
        # Exclude the current pre-synaptic cell from the post-synaptic candidate set
        #normalizing_factor = 0
        normalizing_factor_excitatory_middle_cell = 0
        normalizing_factor_inhibitory_middle_cell = 0
        post_root_ids = backup_assembly_post_root_ids - set([j])
        for i in post_root_ids:
            # Check if pre-synaptic cell 'j' satisfies the condition with post-synaptic cell 'i'
            if cond_function(j, i, A) and j != i:
                pre_idx = mappings_a['pt_root_id_to_connectome_index'][j]
                post_idx = mappings_a['pt_root_id_to_connectome_index'][i]

                # Filter chains where cell A in chain matches pre_idx and cell C matches post_idx
                temporary_chain_results = two_chain_results_array[(two_chain_results_array[:, 0] == pre_idx) & (two_chain_results_array[:, 2] == post_idx)]

                # Add to the normalizing factor the number of realized chains between pre_cell (position A in chain) and post cell (position C in chain)
                #normalizing_factor += len(temporary_chain_results)

                # Continue if no matches are found
                if len(temporary_chain_results) == 0:
                    continue

                for chain in temporary_chain_results:  # Summing over possible intermediate nodes (k's)
                    cell_a, cell_b, cell_c = chain

                    # Classification of the intermediate node (cell B)
                    if classification_map[cell_b] == 'inhibitory':
                        normalizing_factor_inhibitory_middle_cell += 1
                        k = mappings_a['connectome_index_to_root_id'][cell_b]
                        W_out_feed_forward_inhibition_chains[cond_function.__name__][j] = (W_out_feed_forward_inhibition_chains[cond_function.__name__].get(j, 0) +
                        (w_jk[(j, k)]* w_ki[(k,i)]) )

                    elif classification_map[cell_b] == 'excitatory':
                        normalizing_factor_excitatory_middle_cell += 1
                        k = mappings_a['connectome_index_to_root_id'][cell_b]
                        W_out_excitatory_chains[cond_function.__name__][j] = (
                            W_out_excitatory_chains[cond_function.__name__].get(j, 0) +
                            (w_jkE[(j,k)]*w_kiE[(k,i)])
                        )
                        
        if j in W_out_feed_forward_inhibition_chains[cond_function.__name__]:
           W_out_feed_forward_inhibition_chains[cond_function.__name__][j] /= normalizing_factor_inhibitory_middle_cell

        if j in W_out_excitatory_chains[cond_function.__name__]:
            W_out_excitatory_chains[cond_function.__name__][j] /= normalizing_factor_excitatory_middle_cell

In [15]:
# Outbound Summed PSDs

print('\n\nSUMMED OUTBOUND PSDS for Feed-Forward Inhibitory CHAINS normalized based on middle cell')
for c1 in C:
    for c2 in C:
        if c1.__name__ != c2.__name__:
            print(f'\n{c1.__name__} vs {c2.__name__}')
            result = stats.ranksums(list(W_out_feed_forward_inhibition_chains[c1.__name__].values()), list(W_out_feed_forward_inhibition_chains[c2.__name__].values()), alternative='greater')

            print('Greater:')
            print(result)
            if result.pvalue < 0.05:
                plt.figure()
                plt.boxplot([list(W_out_feed_forward_inhibition_chains[c1.__name__].values()), list(W_out_feed_forward_inhibition_chains[c2.__name__].values())])
                plt.title(f'OUTBOUND SUMMED PSDS: {c1.__name__} vs {c2.__name__}')



SUMMED OUTBOUND PSDS for Feed-Forward Inhibitory CHAINS normalized based on middle cell

shared vs disjoint
Greater:
RanksumsResult(statistic=-0.5234929895230153, pvalue=0.6996843858393493)

shared vs shared_no_a
Greater:
RanksumsResult(statistic=-0.12751534261266764, pvalue=0.5507337346707242)

shared vs no_a_a
Greater:
RanksumsResult(statistic=-0.27628324232744655, pvalue=0.6088347370593239)

shared vs a_no_a
Greater:
RanksumsResult(statistic=-0.022371495278761335, pvalue=0.5089241909341238)

disjoint vs shared
Greater:
RanksumsResult(statistic=0.5234929895230153, pvalue=0.3003156141606507)

disjoint vs shared_no_a
Greater:
RanksumsResult(statistic=0.26492022243024727, pvalue=0.39553545975645366)

disjoint vs no_a_a
Greater:
RanksumsResult(statistic=0.16302782918784448, pvalue=0.43524826163304664)

disjoint vs a_no_a
Greater:
RanksumsResult(statistic=0.5829969919512094, pvalue=0.2799476625986654)

shared_no_a vs shared
Greater:
RanksumsResult(statistic=0.12751534261266764, pvalue=0

In [16]:
# Outbound Summed PSDs

print('\n\nSUMMED OUTBOUND PSDS for Excitatory CHAINS normalized based on middle cell')
for c1 in C:
    for c2 in C:
        if c1.__name__ != c2.__name__:
            print(f'\n{c1.__name__} vs {c2.__name__}')
            result = stats.ranksums(list(W_out_excitatory_chains[c1.__name__].values()), list(W_out_excitatory_chains[c2.__name__].values()), alternative='greater')
            print('Greater:')
            print(result)
            if result.pvalue < 0.05:
                plt.figure()
                plt.boxplot([list(W_out_excitatory_chains[c1.__name__].values()), list(W_out_excitatory_chains[c2.__name__].values())])
                plt.title(f'OUTBOUND SUMMED PSDS: {c1.__name__} vs {c2.__name__}')



SUMMED OUTBOUND PSDS for Excitatory CHAINS normalized based on middle cell

shared vs disjoint
Greater:
RanksumsResult(statistic=1.3411370730274548, pvalue=0.08993797627996997)

shared vs shared_no_a
Greater:
RanksumsResult(statistic=0.7895420339517227, pvalue=0.21489763616977792)

shared vs no_a_a
Greater:
RanksumsResult(statistic=-0.384995251754509, pvalue=0.6498795332706171)

shared vs a_no_a
Greater:
RanksumsResult(statistic=0.2757260134719975, pvalue=0.39137925698933224)

disjoint vs shared
Greater:
RanksumsResult(statistic=-1.3411370730274548, pvalue=0.9100620237200301)

disjoint vs shared_no_a
Greater:
RanksumsResult(statistic=0.16598500055174645, pvalue=0.43408437712655834)

disjoint vs no_a_a
Greater:
RanksumsResult(statistic=-1.520302849670743, pvalue=0.9357825609161705)

disjoint vs a_no_a
Greater:
RanksumsResult(statistic=-1.1194615237667185, pvalue=0.8685283516870185)

shared_no_a vs shared
Greater:
RanksumsResult(statistic=-0.7895420339517227, pvalue=0.7851023638302221)

Inbound connectivity

In [17]:
classification_map = cell_table.set_index('connectome_index')['classification_system'].to_dict() # Map to tell us if its Excitatory or Inhibitory 


# Calculate the weighted summed values

backup_assembly_pre_root_ids = assembly_pre_root_ids
backup_assembly_post_root_ids = assembly_post_root_ids




W_in_feed_forward_inhibition_chains = {c.__name__: {} for c in C}
W_in_excitatory_chains = {c.__name__: {} for c in C}


for cond_function in C:
    for i in assembly_post_root_ids:
        # Exclude the current pre-synaptic cell from the post-synaptic candidate set
        #normalizing_factor = 0
        normalizing_factor_excitatory_middle_cell = 0
        normalizing_factor_inhibitory_middle_cell = 0
        pre_root_ids = backup_assembly_pre_root_ids - set([i])
        for j in pre_root_ids:
            # Check if pre-synaptic cell 'j' satisfies the condition with post-synaptic cell 'i'
            if cond_function(j, i, A) and j != i:
                pre_idx = mappings_a['pt_root_id_to_connectome_index'][j]
                post_idx = mappings_a['pt_root_id_to_connectome_index'][i]

                # Filter chains where cell A in chain matches pre_idx and cell C matches post_idx
                temporary_chain_results = two_chain_results_array[(two_chain_results_array[:, 0] == pre_idx) & (two_chain_results_array[:, 2] == post_idx)]

                # Add to the normalizing factor the number of realized chains between pre_cell (position A in chain) and post cell (position C in chain)
                normalizing_factor += len(temporary_chain_results)

                # Continue if no matches are found
                if len(temporary_chain_results) == 0:
                    continue

                for chain in temporary_chain_results:  # Summing over possible intermediate nodes (k's)
                    cell_a, cell_b, cell_c = chain

                    # Classification of the intermediate node (cell B)
                    if classification_map[cell_b] == 'inhibitory':
                        normalizing_factor_inhibitory_middle_cell += 1
                        k = mappings_a['connectome_index_to_root_id'][cell_b]
                        W_in_feed_forward_inhibition_chains[cond_function.__name__][i] = (W_in_feed_forward_inhibition_chains[cond_function.__name__].get(i, 0) +
                        (w_jk[(j, k)]* w_ki[(k,i)]) )

                    elif classification_map[cell_b] == 'excitatory':
                        normalizing_factor_excitatory_middle_cell += 1
                        k = mappings_a['connectome_index_to_root_id'][cell_b]
                        W_in_excitatory_chains[cond_function.__name__][i] = (
                            W_in_excitatory_chains[cond_function.__name__].get(i, 0) +
                            (w_jkE[(j,k)]*w_kiE[(k,i)])
                        )
                        
        if i in W_in_feed_forward_inhibition_chains[cond_function.__name__]:
           W_in_feed_forward_inhibition_chains[cond_function.__name__][i] /= normalizing_factor_inhibitory_middle_cell

        if i in W_in_excitatory_chains[cond_function.__name__]:
            W_in_excitatory_chains[cond_function.__name__][i] /= normalizing_factor_excitatory_middle_cell


In [18]:
# Inbound Summed PSDs

print('\n\nSUMMED INBOUND PSDS for Feed-Forward Inhibitory CHAINS normalized based on middle cell')
for c1 in C:
    for c2 in C:
        if c1.__name__ != c2.__name__:
            print(f'\n{c1.__name__} vs {c2.__name__}')
            result = stats.ranksums(list(W_in_feed_forward_inhibition_chains[c1.__name__].values()), list(W_in_feed_forward_inhibition_chains[c2.__name__].values()), alternative='greater')

            print('Greater:')
            print(result)
            if result.pvalue < 0.05:
                plt.figure()
                plt.boxplot([list(W_in_feed_forward_inhibition_chains[c1.__name__].values()), list(W_in_feed_forward_inhibition_chains[c2.__name__].values())])
                plt.title(f'INBOUND SUMMED PSDS: {c1.__name__} vs {c2.__name__}')



SUMMED INBOUND PSDS for Feed-Forward Inhibitory CHAINS normalized based on middle cell

shared vs disjoint
Greater:
RanksumsResult(statistic=-0.5682359800805379, pvalue=0.7150626283661248)

shared vs shared_no_a
Greater:
RanksumsResult(statistic=-0.10626278551055637, pvalue=0.5423130713853782)

shared vs no_a_a
Greater:
RanksumsResult(statistic=0.4340070084079699, pvalue=0.33214167587785237)

shared vs a_no_a
Greater:
RanksumsResult(statistic=-0.4037985849401142, pvalue=0.6568195814246833)

disjoint vs shared
Greater:
RanksumsResult(statistic=0.5682359800805379, pvalue=0.2849373716338752)

disjoint vs shared_no_a
Greater:
RanksumsResult(statistic=0.3668126156726501, pvalue=0.356879398338411)

disjoint vs no_a_a
Greater:
RanksumsResult(statistic=0.9112027059385568, pvalue=0.18109428927916954)

disjoint vs a_no_a
Greater:
RanksumsResult(statistic=-0.12227087189088336, pvalue=0.5486577501265253)

shared_no_a vs shared
Greater:
RanksumsResult(statistic=0.10626278551055637, pvalue=0.45768

In [19]:
# Inbound Summed PSDs

print('\n\nSUMMED INBOUND PSDS for Excitatory CHAINS normalized based on middle cell')
for c1 in C:
    for c2 in C:
        if c1.__name__ != c2.__name__:
            print(f'\n{c1.__name__} vs {c2.__name__}')
            result = stats.ranksums(list(W_in_excitatory_chains[c1.__name__].values()), list(W_in_excitatory_chains[c2.__name__].values()), alternative='greater')
            print('Greater:')
            print(result)
            if result.pvalue < 0.05:
                plt.figure()
                plt.boxplot([list(W_in_excitatory_chains[c1.__name__].values()), list(W_in_excitatory_chains[c2.__name__].values())])
                plt.title(f'OUTBOUND SUMMED PSDS: {c1.__name__} vs {c2.__name__}')



SUMMED INBOUND PSDS for Excitatory CHAINS normalized based on middle cell

shared vs disjoint
Greater:
RanksumsResult(statistic=0.22780124699975274, pvalue=0.4099003786083848)

shared vs shared_no_a
Greater:
RanksumsResult(statistic=0.13179426029667, pvalue=0.4475735132850578)

shared vs no_a_a
Greater:
RanksumsResult(statistic=-0.7026506773883116, pvalue=0.7588632638507835)

shared vs a_no_a
Greater:
RanksumsResult(statistic=-0.6150602574013412, pvalue=0.7307425365260825)

disjoint vs shared
Greater:
RanksumsResult(statistic=-0.22780124699975274, pvalue=0.5900996213916152)

disjoint vs shared_no_a
Greater:
RanksumsResult(statistic=-0.048131095111188946, pvalue=0.5191941176996614)

disjoint vs no_a_a
Greater:
RanksumsResult(statistic=-0.768723798599197, pvalue=0.7789713530789929)

disjoint vs a_no_a
Greater:
RanksumsResult(statistic=-0.4783326101625762, pvalue=0.683793255143006)

shared_no_a vs shared
Greater:
RanksumsResult(statistic=-0.13179426029667, pvalue=0.5524264867149422)

sh